##### Creation Date:
##### February 19 2022
##### Created By Alperen KOLAMUC

In [1]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics

In [2]:
import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [3]:
import pandas as pd
import numpy as np
import math

##### Prepare


In [4]:
df = pd.read_feather("tmp/bulldozers_1")

In [5]:
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes + 1 # ortak ağız için

In [6]:
def fix_missing(df, col, name, nan_dict, is_train):
    if is_train:
        if is_numeric_dtype(col):
            if pd.isnull(col).sum():
                df[name+"_NA"] = pd.isnull(col)
                nan_dict[name] = col.median()
                df[name] = col.fillna(nan_dict[name])

    else:
        if is_numeric_dtype(col):
            if name in nan_dict:
                df[name+"_NA"] = pd.isnull(col)
                df[name] = col.fillna(nan_dict[name])
            
            else:
                df[name] = col.fillna(df[name].median())

In [7]:
def proc_df(df, y_fld, nan_dict=None, is_train=True):
    df = df.copy()
    y = df[y_fld].values

    df.drop([y_fld], axis=1, inplace=True)

    if nan_dict is None:
        nan_dict = {}
    
    for n, c in df.items():
        fix_missing(df, c, n, nan_dict, is_train)
        numericalize(df, c, n)

    if is_train:
        return df, y, nan_dict
    
    return df, y

In [8]:
def split_train_val(df, n):
    return(df[:n].copy(), df[n:].copy())

In [9]:
n_valid = 12000
n_train = len(df)-n_valid
raw_train, raw_valid = split_train_val(df, n_train)

In [12]:
x_train_raw, y_train_raw, nas = proc_df(raw_train, 'SalePrice')

In [13]:
x_train, y_train, nas = proc_df(raw_train, 'SalePrice')

In [14]:
x_valid, y_valid = proc_df(raw_valid, 'SalePrice', nan_dict=nas, is_train=False)

In [15]:
def rmse(x, y):
    return math.sqrt(((x-y)**2).mean())

In [17]:
def print_score(m):

    print(f"RMSLE of train set {rmse(m.predict(x_train), y_train)}")
    print(f"RMSLE of validation set {rmse(m.predict(x_valid), y_valid)}")
    print(f"R^2 of train set {m.score(x_train, y_train)}")
    print(f"R^2 of validation set {m.score(x_valid, y_valid)}")

#### Farklı Şeyleri Hızlı Şekilde Denemek
* Elimizdeki probleme uygun modeli bulmak için çok şey denememiz gerekebilir
* Bu iteratif süreci hızlı bir hale getirmek için model seçme kısmını subsample alarak yapabiliriz

#### Subset Yaratma

In [18]:
def get_sample(df, n):
    idxs = np.random.permutation(len(df))[:n]
    return idxs, df.iloc[idxs].copy()

* Bütün verini model seçmek için kullanman gerekmez
* En son eğitmek için gerekir
* Validation seti değiştirmek istemiyorum, sadece train içinde subset alacağım


|        | **Train** | **Validation** |
|--------|:---------:|:--------------:|
|        |  **300K** |     **1M**     |
| **M1** |    0.64   |      0.75      |
| **M2** |    0.67   |      0.79      |
| **M3** |    0.58   |      0.68      |
| **M4** |    0.44   |      0.57      |

* Örnekte görüldüğü gibi M2 300K'lık veride `Train`'de en yüksek skoru yapmış. Yine aynı şekilde 1M'luk veride `Validation`'da da yüksek skor yapmış diğer modellere göre. O zaman `Train`'de en yüksek yapan `Validation`'da da yüksek sonuç verecektir. Sırf bunu test etmek için 1M veriyi diğer modellerde de test ederek zaman kaybetmeye gerek yok. `Train`'de en yüksek 2 tane M2 ve M1'i `Validation`'da test etmek daha doğru olacaktır


In [19]:
idxs, x_train = get_sample(x_train, 3000)
y_train = y_train[idxs]

In [24]:
m = RandomForestRegressor(n_estimators=10, n_jobs=-1)
%time m.fit(x_train, y_train)
print_score(m)

CPU times: total: 531 ms
Wall time: 129 ms
RMSLE of train set 0.14249336903498883
RMSLE of validation set 0.3613955219258716
R^2 of train set 0.9572719139586738
R^2 of validation set 0.7374686965119867


In [25]:
m = RandomForestRegressor(n_estimators=30, n_jobs=-1)
%time m.fit(x_train, y_train)
print_score(m)

CPU times: total: 1.81 s
Wall time: 304 ms
RMSLE of train set 0.1226197186777285
RMSLE of validation set 0.341147625856498
R^2 of train set 0.9683593963378302
R^2 of validation set 0.7660622720757944


#### Tüm Veri İle

In [26]:
x_train, y_train, nas = proc_df(raw_train, 'SalePrice')

In [27]:
x_valid, y_valid = proc_df(raw_valid, 'SalePrice', nan_dict=nas, is_train=False)

In [28]:
m = RandomForestRegressor(n_estimators=10, n_jobs=-1)
%time m.fit(x_train, y_train)
print_score(m)

CPU times: total: 1min 43s
Wall time: 19.5 s
RMSLE of train set 0.08993485469865092
RMSLE of validation set 0.2586193068466469
R^2 of train set 0.9829300978330792
R^2 of validation set 0.8655572312564608


In [29]:
m = RandomForestRegressor(n_estimators=30, n_jobs=-1)
%time m.fit(x_train, y_train)
print_score(m)

CPU times: total: 7min 1s
Wall time: 1min 1s
RMSLE of train set 0.07951437555099973
RMSLE of validation set 0.24947229607712493
R^2 of train set 0.9866566074518908
R^2 of validation set 0.8748991647040394
